In [1]:
from glob import glob

import numpy as np
import pandas as pd
from scipy.spatial import distance_matrix
from ortools.graph import pywrapgraph
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [2]:
def list_lines(file_name):
    with open(file_name) as file:
        lines = file.read().splitlines()
    line_list = [[int(n) for  n in ll.split()] for ll in lines]
    return line_list


def set_params(line_list):
    params = {'DRONE_COUNT': line_list[0][2],
              'WT_CAP': line_list[0][4],
              'END_TIME': line_list[0][3],
              }
    return params


def find_wh_lines(line_list):
    wh_count = line_list[3][0]
    wh_endline = (wh_count*2)+4
    return wh_endline


def get_weights(line_list):
    weights = np.array(line_list[2])
    return weights.astype(np.int16)


def get_inventories(line_list):
    wh_endline = find_wh_lines(line_list)
    invs = line_list[5:wh_endline+1:2]
    supply = np.array(invs).transpose()
    return supply.astype(np.int16)


def get_orders(line_list):
    wh_endline = find_wh_lines(line_list)
    demand = np.zeros((line_list[1][0], line_list[wh_endline][0]),
                            dtype=np.int16)
    orders = line_list[wh_endline+3::3]
    for i,ord in enumerate(orders):
        for prod in ord:
            demand[prod, i] += 1
    return demand.astype(np.int16)


def get_locs(line_list):
    wh_endline = find_wh_lines(line_list)
    wh_locs = np.array(line_list[4:wh_endline:2])
    cust_locs = np.array(line_list[wh_endline+1::3])
    return wh_locs.astype(np.int16), cust_locs.astype(np.int16)

In [3]:
files = ['C:/Users/denpo/Desktop/ДроныДипломУсловие/busy_day.in']
line_list = list_lines(files[0])
params = set_params(line_list)
supply = get_inventories(line_list)
demand = get_orders(line_list)
wh_locs, cust_locs = get_locs(line_list)
weights = get_weights(line_list)

In [4]:
def assign_whs(supply, wh_locs, demand, cust_locs):
    assignments = []
    count = 0
    distances = distance_matrix(cust_locs, wh_locs)

    for i in range(400):
        item_count = 0

        start_nodes = np.repeat(np.arange(1250,1260), 1250).tolist()
        end_nodes = np.tile(np.arange(0,1250), 10).tolist()        
        capacities = np.tile(demand[i], 10).tolist()
        costs = np.transpose(distances).ravel().astype(int).tolist()
        supplies = np.negative(demand[i]).tolist() + supply[i].tolist()
                                            
        min_cost_flow = pywrapgraph.SimpleMinCostFlow()

        for s in range(len(start_nodes)):
            min_cost_flow.AddArcWithCapacityAndUnitCost(
                start_nodes[s], end_nodes[s], capacities[s], costs[s]
                )
        for s in range(len(supplies)):
            min_cost_flow.SetNodeSupply(s, supplies[s])

        if min_cost_flow.SolveMaxFlowWithMinCost() == min_cost_flow.OPTIMAL:
            for arc in range(min_cost_flow.NumArcs()):
                if min_cost_flow.Flow(arc) > 0:
                    warehouse = min_cost_flow.Tail(arc) - 1250
                    customer = min_cost_flow.Head(arc)
                    product = i
                    quant = min_cost_flow.Flow(arc)
                    cost = min_cost_flow.UnitCost(arc)
                    assign = [warehouse, customer, product, quant, cost]
                    assignments.append(assign)
                    item_count += quant
        count += item_count
    
    print(f"Products available: {supply.sum()} \n"
            f"Products ordered: {demand.sum()} \n"
            f"Products delivered: {count}")              
    return np.array(assignments)

In [5]:
assignments = assign_whs(supply, wh_locs, demand, cust_locs)
assign_df = pd.DataFrame(assignments, columns=['wh', 'cust', 'prod_',
                                               'quant', 'dist'])

C:\Users\denpo\Anaconda3\lib\site-packages\scipy\spatial\kdtree.py:74: RuntimeWarning: invalid value encountered in power
  return minkowski_distance_p(x, y, p)**(1./p)


Products available: 14576 
Products ordered: 9368 
Products delivered: 9368


In [6]:
def order_orders(df):

    customers = df.cust.unique()
    demand = df.groupby('cust')['quant'].sum()

    locs = np.vstack((cust_locs[customers], wh_locs[0]))

    distances = np.ceil(distance_matrix(locs, locs)).astype(int)

    customer_map = dict(zip(customers, range(len(customers))))

    data = {}
    data['dists'] = distances.tolist()
    data['drone_count'] = 1
    data['warehouse'] = len(locs) - 1

    # мэнэджер индексов маршрутизации
    manager = pywrapcp.RoutingIndexManager(len(data['dists']),
                                           data['drone_count'], data['warehouse'])

    routing = pywrapcp.RoutingModel(manager)

    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['dists'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Setting first solution heuristic
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem
    solution = routing.SolveWithParameters(search_parameters)

    # Get vehicle routes
    routes = []
    for route_nbr in range(routing.vehicles()):
        index = routing.Start(route_nbr)
        route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
        index = solution.Value(routing.NextVar(index))
        route.append(manager.IndexToNode(index))
    routes.append(route[1:-1])

    # Single vehicle approximation
    route = routes[0]

    reverse_dict = {v: k for k,v in customer_map.items()}
    cust_ids = [reverse_dict[r] for r in route]

    df['cust_sort'] = pd.Categorical(df.cust, cust_ids)
    df = df.sort_values('cust_sort')
    return df

In [7]:
def load_drones(df):
    test_wt = 0
    load_wts = []
    df = df.sort_values('cust')
    for i,tup in enumerate(df.itertuples()):
        test_wt += tup.weight
        if test_wt <= params['WT_CAP']:
            load_wt = test_wt
        else:
            load_wt = tup.weight
            test_wt = tup.weight
        load_wts.append(load_wt)

    df['load_weight'] = load_wts
    df['load_tag'] = df.load_weight.eq(df.weight).cumsum()-1
    return df


def set_loads(assignments):
    assign_df = pd.DataFrame(assignments, columns=['wh', 'cust', 'prod_', 'quant', 'dist'])
    assign_df = assign_df.reindex(assign_df.index.repeat(assign_df.quant)) \
                         .reset_index(drop=True) \
                         .assign(quant=1,
                                 weight = lambda x: weights[x.prod_.to_numpy()],
                                 work = lambda x: x.dist * x.weight) \
                         .groupby('wh', as_index=False).apply(load_drones) \
                         .sort_values(['wh', 'cust', 'load_tag']) \
                         .reset_index(drop=True)
    return assign_df


def assign_drones(assign_df):
    wh_work = assign_df.groupby('wh')['work'].sum()
    drones_per_wh = (wh_work/ wh_work.sum()
                         * params['DRONE_COUNT'])
    drone_counts = drones_per_wh.round(0).astype(int)

    if drone_counts.sum() != params['DRONE_COUNT']:
        drone_counts = np.ediff1d(drones_per_wh.cumsum().round(0).astype(int),
                                        to_begin=drone_counts[0])

    drone_whs = np.repeat(np.arange(len(wh_locs)), drone_counts)
    drone_dict = dict(zip(range(params['DRONE_COUNT']), drone_whs))

    drone_assigns = {}
    for k, v in drone_dict.items():
        drone_assigns[v] = drone_assigns.get(v, []) + [k]

    df_list = []
    for grp, df in assign_df.groupby('wh'):
        drone_ids = drone_assigns[df.wh.iloc[0]]
        df['drone_id'] = df.load_tag % len(drone_ids) + min(drone_ids)
        df_list.append(df)

    df_end = pd.concat(df_list)
    return df_end

assign_df = set_loads(assignments)
df_end = assign_drones(assign_df)
df_end = df_end.groupby(['wh', 'cust', 'load_tag', 'drone_id', 'prod_'],
                            as_index=False)['quant'].sum()

df_end

C:\Users\denpo\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,wh,cust,load_tag,drone_id,prod_,quant
0,0,0,0,0,280,1
1,0,0,0,0,299,1
2,0,4,0,0,263,1
3,0,5,1,1,43,1
4,0,6,1,1,179,1
...,...,...,...,...,...,...
9314,9,1240,425,27,350,1
9315,9,1244,425,27,58,1
9316,9,1245,426,28,69,1
9317,9,1246,426,28,58,1


In [8]:
def load_drones_improved(df):
    test_wt = 0
    load_wts = []
    for i,tup in enumerate(df.itertuples()):
        test_wt += tup.weight
        if test_wt <= params['WT_CAP']:
            load_wt = test_wt
        else:
            load_wt = tup.weight
            test_wt = tup.weight
        load_wts.append(load_wt)

    df['load_weight'] = load_wts
    df['load_tag'] = df.load_weight.eq(df.weight).cumsum()-1
    return df


def reset_loads(df_end_reordered):
    df_end_reordered = df_end_reordered.reset_index(drop=True) \
                                 .assign(weight = lambda x: weights[x.prod_.to_numpy()] * x.quant) \
                                 .groupby('wh', as_index=False).apply(load_drones_improved)
    return df_end_reordered


def assign_drones(df_end_reordered):
    df_list = []
    for grp, df in df_end_reordered.groupby('wh'):
        drone_ids = df.wh.iloc[0]
        df['drone_id'] = df.load_tag % df.drone_id.nunique() + min(df.drone_id)
        df_list.append(df)

    df_end = pd.concat(df_list)
    return df_end


df_end_reordered = df_end.groupby('wh').apply(order_orders) \
                         .pipe(reset_loads) \
                         .pipe(assign_drones)

df_end_reordered

C:\Users\denpo\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,wh,cust,load_tag,drone_id,prod_,quant,cust_sort,weight,load_weight
0,0,777,0,0,277,1,777,100,100
1,0,777,0,0,62,1,777,2,102
2,0,800,0,0,179,1,800,26,128
3,0,890,0,0,149,1,890,21,149
4,0,890,0,0,310,1,890,39,188
...,...,...,...,...,...,...,...,...,...
9314,9,155,432,26,301,1,155,58,156
9315,9,155,433,27,181,1,155,69,69
9316,9,240,433,27,116,1,240,68,137
9317,9,475,433,27,111,1,475,63,200


In [14]:
num = 0;
for i in df_end_reordered['quant']:
    if i>1:
        num=num+1
num

49